<a href="https://colab.research.google.com/github/Rishit-dagli/GLOM-TensorFlow/blob/example/examples/glom_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GLOM Example

This notebook shows the the process of using the [glom-tf](https://github.com/Rishit-dagli/GLOM-TensorFlow) Python package. 
This Python package attempts to implement GLOM in TensorFlow, which allows advances made by several different groups 
transformers, neural fields, contrastive representation learning, distillation and capsules to be combined. This was 
suggested by Geoffrey Hinton in his paper 
["How to represent part-whole hierarchies in a neural network"](https://arxiv.org/abs/2102.12627).

If you find this useful please consider giving a ⭐ to [the repo](https://github.com/Rishit-dagli/GLOM-TensorFlow).


## A bit about GLOM

The GLOM architecture is composed of a large number of columns which
all use exactly the same weights. Each column is a stack of spatially local
autoencoders that learn multiple levels of representation for what is happening
in a small image patch. Each autoencoder transforms the embedding at one level
into the embedding at an adjacent level using a multilayer bottom-up encoder
and a multilayer top-down decoder. These levels correspond to the levels in a
part-whole hierarchy.

![](https://github.com/Rishit-dagli/GLOM-TensorFlow/blob/main/images/interactions.png?raw=true)
<p align="center">
<small>Interactions among the 3 levels in one column</small>
</p>

An example shared by the author was as an example when show a face image, a single column might converge on embedding 
vectors representing a nostril, a nose, a face, and a person.

At each discrete time and in each column separately, the embedding at a
level is updated to be the weighted average of:
- bottom-up neural net acting on the embedding at the level below at the previous time
- top-down neural net acting on the embedding at the level above at the previous time
- embedding vector at the previous time step
- attention-weighted average of the embeddings at the same level in nearby columns at the previous time

For a static image, the embeddings at a level should settle down over time to produce similar vectors.

![](https://github.com/Rishit-dagli/GLOM-TensorFlow/blob/main/images/embeddings.png?raw=true)
<p align="center">
<small>A picture of the embeddings at a particular time</small>
</p>

## Setup

In [1]:
import tensorflow as tf

### Install the package

In [2]:
!pip install glom-tf

## Create a Glom class

In [3]:
from glomtf import Glom

In [4]:
model = Glom(dim = 512,
             levels = 5,
             image_size = 224,
             patch_size = 14)

In [5]:
img = tf.random.normal([1, 3, 224, 224])
all_levels = model(img, iters = 12, return_all = True) # (13, 1, 256, 5, 12)
# 13 - time

# top level outputs after iteration 6
top_level_output = all_levels[7, :, :, -1] # (1, 256, 512)
# 1 - batch
# 256 - patches
# 512 - dimensions

In [6]:
all_levels.shape

TensorShape([13, 1, 256, 5, 512])

In [7]:
top_level_output.shape

TensorShape([1, 256, 512])